In [ ]:
%pip install firebase_admin

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, storage
import io
import os
from io import StringIO

from google.cloud import speech_v1p1beta1

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="john-depotube-personal-be14008820a6.json"

In [4]:
# Initialize Firestore DB
cred = credentials.Certificate('firebasekey.json')
default_app = initialize_app(cred)
db = firestore.client()
sampleCollection_ref = db.collection('sampleCollection')

In [5]:
doc_ref = sampleCollection_ref.document(u'Shea Jensen-9-17-19').get()

In [6]:
d = doc_ref.to_dict()
d['firmName']

'Hollins & Levy'

In [22]:
%pip install requests_toolbelt

Note: you may need to restart the kernel to use updated packages.


In [27]:
import firebase

stor = firebase.storage

In [30]:
dir(stor.bucket)

['Blob',
 'Bucket',
 'BucketACL',
 'DefaultObjectACL',
 'Iterator',
 'NotFound',
 '_BlobIterator',
 '_PropertyMixin',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_rfc3339_to_datetime',
 '_scalar_property',
 'copy',
 'six']

In [24]:
import firebase
import python_jwt
stor = firebase.storage()
pathReference = stor.ref('testseq3114to200.txt')

TypeError: 'module' object is not callable

In [7]:
d

{'firmName': 'Hollins & Levy',
 'clientName': 'Shea Jensen',
 'date': '09-17-19',
 'videoURL': 'https://storage.googleapis.com/john-depotube-personal.appspot.com/testseq3115to200.wav',
 'transcriptionURL': 'https://storage.googleapis.com/john-depotube-personal.appspot.com/testseq3115to200.txt'}

In [31]:
# Use custom bucket
bucket = storage.bucket('john-depotube-personal.appspot.com')

audioPath = "Shea Jensen-9-17-19-1of3_data/testseq3115to200.wav" # path to local file being uploaded
audioBlob = bucket.blob("testseq3115to200.wav") # create Blob object with name of file in firebase
audioBlob.upload_from_filename(audioPath) # do the actual upload

In [36]:
import time

blob_expiry = int(time.time() + (60 * 60 * 24 * 7)) # set for 7 days
signedURL = audioBlob.generate_signed_url(expiration=blob_expiry)

In [37]:
signedURL

'https://storage.googleapis.com/john-depotube-personal.appspot.com/testseq3115to200.wav?Expires=1594242455&GoogleAccessId=firebase-adminsdk-glkfa%40john-depotube-personal.iam.gserviceaccount.com&Signature=NuXCjdmDDfTZAuLrD6x4Me9FnIy1MFHjW2axjUCEdrl%2F0uGsP6bAZ9NASOOL9kvKDF8aWa%2FLOAkjW1S490MetkLRrrclV49mY5aje1vKDKEreT%2FmC4S%2F%2FuWCEGpxKpUWZ%2FJG0vHuRWSHPoPp1AJXC%2BJCjBcOeNr2aSP%2Fidbiy60RN8wwT6OBSjwIjuGzXfSw4mJoogIFc0OuI3gmAHTiRSEjgjeGToiuIjfZSSN95Lq0ZZNssbRk%2B%2BYSHoy9FQi1Yl431FevkMwZcTa6ZRP%2FkHEOYxezyGOli34EXa%2BsrH0UwhQt5nqBvn%2FmuZVrxWH24ieDekxfhjRoqOlFQt3XWw%3D%3D'

In [6]:
audioBlob.public_url

'https://storage.googleapis.com/john-depotube-personal.appspot.com/testseq3115to200.wav'

In [ ]:
# Google Speech to Text needs Google Cloud Storage URI 
object_uri = "gs://john-depotube-personal.appspot.com/testseq3115to200.wav"

In [ ]:
# local_file_path = "Shea Jensen-9-17-19-1of3_data/testseq3115to200.wav"

client = speech_v1p1beta1.SpeechClient()

enable_word_confidence = True

# If enabled, each word in the first alternative of each result will be
# tagged with a speaker tag to identify the speaker.
enable_speaker_diarization = True
enable_wordtime_offsets = True

# The language of the supplied audio
language_code = "en-US"
config = {
        "enable_speaker_diarization": enable_speaker_diarization,
        "enable_word_confidence": enable_word_confidence,
        "language_code": language_code,
        #"enableWordTimeOffsets": enable_wordtime_offsets,
}
# with io.open(local_file_path, "rb") as f:
#    content = f.read()
audio = {"uri": object_uri}

    

In [ ]:
operation = client.long_running_recognize(config, audio)

print(u"Waiting for operation to complete...")
response = operation.result()

In [ ]:
# response = client.recognize(config, audio)
# The first result includes confidence levels per word
result = response.results[0]
# First alternative is the most probable result
alternative = result.alternatives[0]
print(u"Transcript: {}".format(alternative.transcript))
# Print the confidence level of each word
for word in alternative.words:
    print(u"Word: {}".format(word.word))
    print(u"Confidence: {}".format(word.confidence))
    print(u"Speaker tag: {}".format(word.speaker_tag))
    

In [ ]:
# response = client.recognize(config, audio)

In [ ]:
# alternative.words
result

In [ ]:


soutput = StringIO()
for i in range(0, response.results.__len__()-1):
    alternative = response.results[i].alternatives[0]
    soutput.write(alternative.transcript + '\n')
    print (alternative.transcript)

# print soutput.getvalue()

transcriptionBlob = bucket.blob("testseq3115to200.txt") # name of file in firebase
transcriptionBlob.upload_from_string(soutput.getvalue()) # do the actual upload

In [ ]:
transcriptionBlob = bucket.blob("testseq3115to200.txt") # name of file in firebase
transcriptionBlob.upload_from_string(soutput.getvalue()) # do the actual upload

In [ ]:
data = {

    u'firmName': u'Hollins & Levy',
    u'clientName': u'Shea Jensen',
    u'date': '09-17-19', # datetime.datetime.now(),
    u'videoURL': audioBlob.public_url,
    u'transcriptionURL': transcriptionBlob.public_url,
}

sampleCollection_ref.document(u'Shea Jensen-9-17-19').set(data)